In [ ]:
import numpy as np, torch, scipy.io as io, os, matplotlib.pyplot as plt, h5py, PIL.Image as Image, pathlib
import sys
sys.path.insert(0, "../..")
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

import utils.helper_functions as helper
import utils.diffuser_utils as diffuser_utils
import dataset.preprocess_data as prep_data
import train

import studies.experimental_results.experimental_predict_utils as exp_utils

%load_ext autoreload
%autoreload 2

%matplotlib inline


## Notebook to reproduce experimental results figures

We gather a number of compressed measurements from indoor and outdoor scenes using our experimental prototype, here we use a learned model to generate reconstructions for each of these measurements. Some of the results are selected for the paper figures, others are included as additional examples and failure modes.

All image outputs are false-color projections of 3d hyperspectral volumes.


In [ ]:
SAMPLE_DATA_DIRPATH = os.path.join(pathlib.Path().resolve(), "data") # TODO: script to get this from google drive. These have some preprocessing applied already
OUTPUTS_DIRPATH = os.path.join(pathlib.Path().resolve(), "outputs")
CONFIGURATION_FILE_DIRPATH = os.path.join(pathlib.Path().resolve(), "configs")

def plot_measurement_stack(exp_meas):
    fig, ax = plt.subplots(1, len(exp_meas), figsize = (4*len(exp_meas), 4))
    fig.set_dpi(180)
    for i, meas in enumerate(exp_meas):
        ax[i].imshow(meas, cmap='gray')
        ax[i].set_title(f"Focus level: {i}")
        ax[i].axis('off')
    plt.show()

LEARNED_MODEL_CONFIG_NAME = "defocuscam_learned.yml"
os.listdir(SAMPLE_DATA_DIRPATH)

In [ ]:
config = helper.read_config(os.path.join(CONFIGURATION_FILE_DIRPATH, LEARNED_MODEL_CONFIG_NAME))
model = train.get_model(config, config['device'])

In [ ]:
def reconstruct_experimental(sample_name: str, config=config, model=model):
    """ Pick a name from above and pass it into this function to run and save a reconstruction"""
    config_measurement_kwargs = {
        "image_center": config['image_center'],
        "image_dims": config['patch_crop'],
        'patch_size': config['patch_size'],
        'blur_levels': config['forward_model_params']['stack_depth'],
        'blurstride': config['forward_model_params']['psf']['stride'],
        'blur_start_idx': config['forward_model_params']['psf'].get("idx_offset", 0)
    }
    measurements_dir = os.path.join(SAMPLE_DATA_DIRPATH, sample_name)
    measurements = exp_utils.get_experimental_measurement_stack(
        measurements_dir=measurements_dir,
        **config_measurement_kwargs
    )
    plot_measurement_stack(measurements)

    recon = exp_utils.predict_experimental(measurements, model.model2)
    recon_path = exp_utils.save_reconstructed_measurement(
        recon,
        out_base_path=OUTPUTS_DIRPATH,
        checkpoint_path=config['checkpoint_dir'],
        measurement_path=measurements_dir,
    )
    recon_fc, recon_fc_path = exp_utils.save_reconstructed_fc_image(
        recon,
        out_base_path=OUTPUTS_DIRPATH,
        checkpoint_path=config['checkpoint_dir'],
        measurement_path=measurements_dir,
    )
    display(recon_fc)
    return recon_path, recon_fc_path, recon_fc

### Figure measurements: indoor

In [ ]:
# Indoor scene 1: mushroom knife pin
reconstruct_experimental("mushroom_knife")

In [ ]:
# Indoor scene 2: color_palette
reconstruct_experimental("color_palette")

In [ ]:
# Indoor scene 3: USAF negative resolution target
reconstruct_experimental("usaf_negative")

### Figure measurements: outdoor scenes -- success cases
Examples where outdoor scenes were measured successfully

In [ ]:
# Outdoor scene 1: umbrella on soda hall
reconstruct_experimental("outside_eight2")

In [ ]:
# Outdoor scene 2: hostel symbol
reconstruct_experimental("outside_six")

In [ ]:
# Outdoor scene 3: campanile long range
reconstruct_experimental("outside_nine2")

### Figure measurements: outdoor scenes -- failure cases
Examples where outdoor scenes were measured, but reconstructions failed due to shaking of the camera, improper manual defocus application, or movement in the scene between measurements.

In [ ]:
LEARNED_MODEL_2_PSF_CONFIG_NAME = "defocuscam_learned_2_psf.yml"
config_2_psf = helper.read_config(os.path.join(CONFIGURATION_FILE_DIRPATH, LEARNED_MODEL_2_PSF_CONFIG_NAME))
model_2_psf = train.get_model(config, config['device'])

In [ ]:
reconstruct_experimental("outside_two", config=config_2_psf, model=model_2_psf)

In [ ]:
reconstruct_experimental("outside_three", config=config_2_psf, model=model_2_psf)

In [ ]:
reconstruct_experimental("outside_four", config=config_2_psf, model=model_2_psf)